## Machine Learning Project 1

In [1]:
#Imprt important libarires
import numpy as np
import os,cv2,math,glob,random
import scipy.io as sio
from math import cos, sin
from pathlib import Path
import pandas as pd
import mediapipe
import warnings
warnings.filterwarnings('ignore')

#Importing Models
from sklearn.svm import SVR
from sklearn.linear_model import Ridge

#preprocessing
from sklearn.preprocessing import MinMaxScaler

#Importing for splitting data
from sklearn.model_selection import train_test_split

#Import model for multioutput 
from sklearn.multioutput import MultiOutputRegressor

#Importing metrics to test the model
from sklearn.metrics import mean_squared_error, r2_score


In [2]:
def draw_axis(img, pitch,yaw,roll, tdx=None, tdy=None, size = 100):

    yaw = -yaw
    if tdx != None and tdy != None:
        tdx = tdx
        tdy = tdy
    else:
        height, width = img.shape[:2]
        tdx = width / 2
        tdy = height / 2

    # X-Axis pointing to right. drawn in red
    x1 = size * (cos(yaw) * cos(roll)) + tdx
    y1 = size * (cos(pitch) * sin(roll) + cos(roll) * sin(pitch) * sin(yaw)) + tdy

    # Y-Axis | drawn in green
    #        v
    x2 = size * (-cos(yaw) * sin(roll)) + tdx
    y2 = size * (cos(pitch) * cos(roll) - sin(pitch) * sin(yaw) * sin(roll)) + tdy

    # Z-Axis (out of the screen) drawn in blue
    x3 = size * (sin(yaw)) + tdx
    y3 = size * (-cos(yaw) * sin(pitch)) + tdy

    cv2.line(img, (int(tdx), int(tdy)), (int(x1),int(y1)),(0,0,255),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x2),int(y2)),(0,255,0),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x3),int(y3)),(255,0,0),2)

    return img

In [3]:
#loops to get the name of the image
img_names = []
img_itr = []
for file_name in sorted(os.listdir('M:\AFLW2000\\')):
        if file_name.endswith(".jpg"):
            img_names.append(file_name)
for i in img_names:
    img_itr.append(i[0:10])
img_itr[0]



'image00002'

In [4]:
# Get the training data for the model
X = []
Y = []
Labels = []

for i in img_itr:
    faceModule = mediapipe.solutions.face_mesh
    # loading image and its correspinding mat file
    with faceModule.FaceMesh(static_image_mode=True) as faces:
        # loading the image
        image = cv2.imread('M:\AFLW2000\\'+i+'.jpg')
        # processing the face to extract the landmark points (468 point) for each x,y,z
        results = faces.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        if results.multi_face_landmarks: 
            landmarks = results.multi_face_landmarks[0]
            x_list = []
            y_list = []
            for point in landmarks.landmark:
                x = point.x * image.shape[1]
                y = point.y * image.shape[0]
                x_list.append(x)
                y_list.append(y)
            X.append(x_list)
            Y.append(y_list)
    
            mat_file = sio.loadmat('M:\AFLW2000\\'+i+'.mat')
            Labels.append(mat_file['Pose_Para'][0][:3])
X = np.array(X)
Y = np.array(Y)
Labels = np.array(Labels)
    

                    



In [5]:
#Normalize the points and center them with respect to a certain point(Nose point)
X_norm = X - X[:,1].reshape(-1, 1)
Y_norm = Y - Y[:,1].reshape(-1, 1)
#To scale with respect to the face 
d = np.linalg.norm(np.array((X[:, 10],Y[:, 10])) - np.array((X[:, 171],Y[:, 171])),axis = 0).reshape(-1,1)
X_norm = X_norm / d
Y_norm = Y_norm / d

In [6]:
#Concatinate the Features
Features = np.hstack([X_norm, Y_norm])
Features.shape, Labels.shape


((1853, 936), (1853, 3))

In [7]:
#Splitting the data 
x_train, x_test, y_train, y_test = train_test_split(Features, Labels, test_size=0.2, random_state=42)
#Select the model 
mod = MultiOutputRegressor(SVR())
#mod = MultiOutputRegressor(Ridge())
#mm_scl = MinMaxScaler()
#X_train = mm_scl.fit_transform(x_train)
#X_test = mm_scl.transform(x_test)

#Fitting the data to train the model
mod.fit(x_train, y_train)
pred = mod.predict(x_test)
print(f"R2 Score by SVR:", mod.score(x_test, y_test)*100, "%")


R2 Score by SVR: 85.04697182103729 %


In [8]:
print("Mean square error by SVR:", mean_squared_error(y_test, pred))

Mean square error by SVR: 0.01581447491672847


In [9]:
#Function to take the image and the model and call the draw_axis func to show the image with tha axis 
def img_lnd(image, model, scaler=None):
    #landmarks=[]
    X=[]
    Y=[]
    #poses=[]
    faceModule = mediapipe.solutions.face_mesh
    with faceModule.FaceMesh(static_image_mode=True) as face:
            points = face.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            if points.multi_face_landmarks: 
                landmark=points.multi_face_landmarks[0]
                x_list=[]
                y_list=[]
                for points in landmark.landmark:
                    x= points.x * image.shape[1]
                    y= points.y * image.shape[0]
                    
                    # X_features
                    x_list.append(x)
                    # Y_features
                    y_list.append(y)
            
                X.append(np.array(x_list))
                Y.append(np.array(y_list))

            
    X = np.array(X)
    Y = np.array(Y)
    X_norm = X - X[:,1].reshape(-1, 1)
    Y_norm = Y - Y[:,1].reshape(-1, 1)
    d = np.linalg.norm(np.array((X[:, 10],Y[:, 10])) - np.array((X[:, 171],Y[:, 171])),axis = 0).reshape(-1,1)
    X_norm = X_norm / d
    Y_norm = Y_norm / d

    
    nose_x = X[:,1]
    nose_y = Y[:,1]


    features = np.hstack([X_norm,Y_norm])
    pd = model.predict(features)
    cx = draw_axis(image, pd[0][0], pd[0][1], pd[0][2], nose_x, nose_y)
    print(pd)
    print('\n')

    return cx


    


In [10]:
image = cv2.imread(r"C:\Users\eng_m\OneDrive\Desktop\b21e703a7684d7edea23c99b0aaaa4fe.jpg")

In [16]:
#image = cv2.imread('M:\AFLW2000\\' + 'image00056' + '.jpg')

In [11]:
c = img_lnd(image, mod)
cv2.imwrite("trial_60.jpg", c)


[[0.34432058 0.84784041 0.44977292]]




True

In [12]:
# Take the vedio to divide it into frames and then draw the axis on each frame by calling the below functions
cap = cv2.VideoCapture(r"C:\Users\eng_m\OneDrive\Pictures\Camera Roll\WIN_20230220_13_44_07_Pro.mp4")
frame_rate = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(r"M:\New folder (5)\vedio_100.mp4", fourcc, frame_rate, (width, height))

while True:
        try:
        # Read the next frame from the video
            ret, frame = cap.read()
            if not ret:
              break
            frame = cv2.flip(frame,1)
            frame = img_lnd(frame, mod)
            # cv2_imshow(frame)
            out.write(frame)
        except Exception as error:
         print(error)
        
          
  
cap.release()
out.release()

[[ 0.01755836 -0.01000972  0.05863266]]


[[-0.00303649 -0.00909009  0.05997294]]


[[-0.00405859  0.00615003  0.06867669]]


[[ 0.00564012 -0.00519706  0.05700461]]


[[0.01956735 0.00321162 0.05964645]]


[[-0.00629297 -0.00293319  0.0629619 ]]


[[ 0.01162224 -0.0083817   0.06117613]]


[[ 0.00406298 -0.00356113  0.06080005]]


[[ 0.0164671  -0.01092846  0.06064259]]


[[ 0.01387113 -0.00316052  0.0586329 ]]


[[ 0.01920061 -0.00574616  0.05451086]]


[[-0.00203546  0.00026998  0.05467341]]


[[ 0.03028372 -0.0096963   0.05589671]]


[[ 0.02415904 -0.0072987   0.05742869]]


[[ 0.0294003  -0.01289297  0.06020761]]


[[ 0.00951555 -0.00860548  0.05674959]]


[[ 0.02440326 -0.01603536  0.05834925]]


[[ 0.02548096 -0.01156179  0.05146365]]


[[ 0.01497498 -0.00480441  0.05360439]]


[[ 0.0179715  -0.010389    0.06009293]]


[[ 0.01934033 -0.00617543  0.05543609]]


[[ 0.02909716 -0.0078329   0.05373508]]


[[ 0.02104829 -0.01042594  0.0582942 ]]


[[ 0.02690993 -0.00064855  0.0554182 